<a href="https://colab.research.google.com/github/erwanBellon/2025_ML_EES/blob/main/project/code/model_habitat_amount_3000_v0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part I: Setup



In [1]:

# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Is this notebook running on Colab or Kaggle?
IS_COLAB = "google.colab" in sys.modules
IS_KAGGLE = "kaggle_secrets" in sys.modules

# Cloning repo or fetch latest changes and path management
!git clone https://github.com/erwanBellon/2025_ML_EES.git
%cd /content/2025_ML_EES
!git pull

import os
from pathlib import Path

# Move into the project directory
%cd /content/2025_ML_EES/project/code
print("Current working directory:", Path.cwd())

# Define main project dir and outputs
PROJECT_ROOT_DIR = Path.cwd().parent       # -> /content/2025_ML_EES/project
OUTPUTS_PATH = PROJECT_ROOT_DIR / "outputs"
OUTPUTS_PATH.mkdir(parents=True, exist_ok=True)
print("Outputs will be saved to:", OUTPUTS_PATH)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model, Input
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU detected. CNNs can be slow without GPU.")

# Common imports
import pandas as pd
import numpy as np
!pip install rasterio
import rasterio
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix


# To make notebook reproducible
np.random.seed(42)
tf.random.set_seed(42)

# For plots
import matplotlib.pyplot as plt
%matplotlib inline

# Load Tensorboard
%load_ext tensorboard

Cloning into '2025_ML_EES'...
remote: Enumerating objects: 988, done.
remote: Counting objects: 100% (236/236), done.
remote: Compressing objects: 100% (174/174), done.
remote: Total 988 (delta 181), reused 62 (delta 62), pack-reused 752 (from 3)
Receiving objects: 100% (988/988), 257.25 MiB | 10.97 MiB/s, done.
Resolving deltas: 100% (649/649), done.
Updating files: 100% (364/364), done.
/content/2025_ML_EES
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 5 (delta 3), reused 0 (delta 0), pack-reused 0 (from 0)
Unpacking objects: 100% (5/5), 4.33 KiB | 1.08 MiB/s, done.
From https://github.com/erwanBellon/2025_ML_EES
 + 5580c1c...8f37531 main       -> origin/main  (forced update)
hint: You have divergent branches and need to specify how to reconcile them.
hint: You can do so by running one of the following commands sometime before
hint: your next pull:
hint: 
hint:   git config pull.rebase fa

In [2]:
print(Path.cwd())

/content/2025_ML_EES/project/code


# Part 2: Load files
## 2.1: Load the presences and absences landcover crops

In [3]:
# --- Load table data ---
rds_path = Path("../data/Table_preds/function_3_100.rds")
!pip install pyreadr
import pyreadr
result = pyreadr.read_r(rds_path)
table_df = result[None]

# Select relevant columns
feature_df = table_df[['MAP','MAT','habitat_amount_3000']].astype(float)

# Labels come from presence/absence index lists
# Load image index files (no images) (just to get the indices that I need from my datatable)
presences_path = Path("../data/cropped_landcover/presences")
absences_path = Path("../data/cropped_landcover/absences")

def extract_indices(folder):
    tif_files = list(folder.glob("*.tif"))
    indices = []
    for tif in tif_files:
        idx = int(tif.stem.split("_")[-1])
        indices.append(idx)
    return np.array(indices)

indices_pres = extract_indices(presences_path)
indices_abs = extract_indices(absences_path)

# Build label vector
y = np.concatenate([
    np.ones(len(indices_pres)),
    np.zeros(len(indices_abs))
]).astype(np.float32)

# Extract corresponding rows
feature_all = feature_df
features = feature_all.iloc[np.concatenate([indices_pres, indices_abs])]
features = features.reset_index(drop=True)

# Normalize features 0–1
features = (features - features.min()) / (features.max() - features.min())
features = features.to_numpy(np.float32)

# Train/valid/test split
X_train, X_temp, y_train, y_temp = train_test_split(
    features, y, test_size=0.2, random_state=42, stratify=y
)
X_valid, X_test, y_valid, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.2/776.2 kB 11.6 MB/s eta 0:00:00


Note that there is some data leakage here as I pre-process part of my test set with the data of the training for normalizing the features. I don't have the time to change that... I've realised to late

## 2.2 Preprocess the data

In [4]:
BATCH_SIZE = 8

def preprocess_table(table, label):
    table = tf.cast(table, tf.float32)
    label = tf.cast(label, tf.float32)
    return table, label

train_ds = (
    tf.data.Dataset.from_tensor_slices((X_train, y_train))
      .shuffle(1000)
      .map(preprocess_table)
      .batch(BATCH_SIZE)
      .prefetch(tf.data.AUTOTUNE)
)

valid_ds = (
    tf.data.Dataset.from_tensor_slices((X_valid, y_valid))
      .map(preprocess_table)
      .batch(BATCH_SIZE)
      .prefetch(tf.data.AUTOTUNE)
)

test_ds = (
    tf.data.Dataset.from_tensor_slices((X_test, y_test))
      .map(preprocess_table)
      .batch(BATCH_SIZE)
      .prefetch(tf.data.AUTOTUNE)
)

train_ds.name = "Training"
valid_ds.name = "Validation"
test_ds.name = "Test"


# 3. ANN model Training

In [5]:
# Image branch: replace this branch by the proportion of forest

# Table branch
table_input = keras.Input(shape=(3,), name="table_input")
y_branch = layers.Dense(16, activation="relu")(table_input)
y_branch = layers.Dense(8, activation="relu")(y_branch)

#Previously CONCAT branch
z = layers.Dense(32, activation="relu")(y_branch)
z = layers.Dropout(0.4)(z)
z = layers.Dense(8, activation="relu")(z)
final_output = layers.Dense(1, activation="sigmoid", name="suitability")(z)

model = keras.Model(inputs=table_input, outputs=final_output)

model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        loss="binary_crossentropy",
        metrics=[tf.keras.metrics.AUC(name="auc")]
    )

model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ table_input (InputLayer)        │ (None, 3)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │            64 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 8)              │           264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ suitability (Dense)             │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 761 (2.97 KB)

 Trainable params: 761 (2.97 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
stop_early = keras.callbacks.EarlyStopping(
    monitor="val_auc", patience=15, mode="max", restore_best_weights=True
)

history = model.fit(
    train_ds,
    validation_data=valid_ds,
    epochs=50,
    callbacks=[stop_early]
)

Epoch 1/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 5s 27ms/step - auc: 0.4486 - loss: 0.6931 - val_auc: 0.7479 - val_loss: 0.6778
Epoch 2/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - auc: 0.6491 - loss: 0.6770 - val_auc: 0.7875 - val_loss: 0.6549
Epoch 3/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - auc: 0.7729 - loss: 0.6401 - val_auc: 0.7979 - val_loss: 0.6123
Epoch 4/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - auc: 0.7552 - loss: 0.6097 - val_auc: 0.8000 - val_loss: 0.5801
Epoch 5/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - auc: 0.7821 - loss: 0.5712 - val_auc: 0.8042 - val_loss: 0.5558
Epoch 6/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - auc: 0.7047 - loss: 0.5943 - val_auc: 0.8063 - val_loss: 0.5467
Epoch 7/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - auc: 0.8064 - loss: 0.5139 - val_auc: 0.8042 - val_loss: 0.5379
Epoch 8/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - auc: 0.8195 - loss: 0.5347 - val_auc: 0.8042 - val_loss: 0.5352
Epoch 9/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - auc: 0.7332 - 

In [7]:
test_loss, test_auc = model.evaluate(test_ds)
print("\n Final Test AUC =", test_auc)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - auc: 0.6689 - loss: 0.6280 

 Final Test AUC = 0.6854166984558105


MaxTSS is not implemented in keras. I'll compute
it there:

In [8]:
def compute_maxTSS(y_true, y_prob):
    thresholds = np.linspace(0, 1, 200)
    best_tss = -2
    best_threshold = None

    for t in thresholds:
        preds = (y_prob >= t).astype(int)
        tn, fp, fn, tp = confusion_matrix(y_true, preds).ravel()

        sensitivity = tp / (tp + fn + 1e-9)
        specificity = tn / (tn + fp + 1e-9)
        tss = sensitivity + specificity - 1

        if tss > best_tss:
            best_tss = tss
            best_threshold = t

    return best_tss, best_threshold


In [9]:
y_prob = model.predict(test_ds)
y_true = np.concatenate([y for (x, y) in test_ds])
max_tss, best_threshold = compute_maxTSS(y_true, y_prob)

print("MaxTSS:", max_tss)
print("Optimal threshold:", best_threshold)


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
MaxTSS: 0.4999999999
Optimal threshold: 0.18592964824120603


Save my best model and mannually load it in GitHub

In [10]:
from google.colab import files
model.save("HA_3000_bestModel.keras")
files.download("HA_3000_bestModel.keras")



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>